In [53]:
import csv
import numpy as np
import ast
import json
import os
import sys
import re
import datetime
import networkx as nx
from textblob import TextBlob
import collections
import pandas as pd
import tldextract
import requests
from datetime import datetime
from datetime import timedelta
from urllib.parse import urlparse, urlsplit, urlunsplit
from time import sleep
from datetime import date
from networkx import density

In [2]:
def featuresExtraction(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    features = []
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    
    return np.array(features)

In [3]:
def labelExtraction(label_file):
#     csv_file = open(file)
#     csv_reader = csv.reader(csv_file, delimiter = ' ')
#     labels = []
#     for row in csv_reader:
#         labels.append(row[0])
    labels = np.genfromtxt(label_file,dtype=None)
    return labels

In [4]:
def reduction(array, key):
    reducedArray = []
    for i, j in enumerate(array):
        if len(reducedArray) == 0:
            reducedArray.append(j)
        else:
            count = 0
            for k in range(len(reducedArray)):
                if a[k] != b[i]:
                    count = count + 1
            
                if count == len(a):
                    a.append(b[i])
    
    for i, j in enumerate(convertedArray):
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            features.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            features.append(j['extended_tweet']['full_text'])
        else:
            features.append(j['text'])
    return reducedArray

In [5]:
def getCredentials():
    app_key = "jGVy1ilV55A7P24eJHuousM1b"
    app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
    oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
    oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
    
    return app_key, app_secret, oauth_token, oauth_token_secret

In [6]:
def featuresExtraction2(features_file):
    convertedArray = json.load(open(features_file, 'r'))
    final = []
    textFeatures = []
    
    for i, j in enumerate(convertedArray):
        tempDict = {}
        tempDict['id'] = j['id']
        tempDict['date'] = j['created_at']
        tempDict['username'] = j['user']['screen_name']
        tempDict['hashtags'] = j['entities']['hashtags']
        tempDict['user_mentions'] = j['entities']['user_mentions']
        tempDict['favorites'] = j['favorite_count']
        
        if 'retweeted_status'in j and 'extended_tweet' in j['retweeted_status']:
            textFeatures.append(j['retweeted_status']['extended_tweet']['full_text'])
        elif 'retweeted_status' not in j and 'extended_tweet' in j:
            textFeatures.append(j['extended_tweet']['full_text'])
        else:
            textFeatures.append(j['text'])
        
        final.append(tempDict)
    
    return final, textFeatures

In [7]:
def authorization(key):
    if key == 'e' or 'E':
        app_key = "jGVy1ilV55A7P24eJHuousM1b"
        app_secret = "wrPOxGirrXEUQsU7Pk0jqaqt7nLYq10TmR0E6QdZPSlvvrktE8"
        oauth_token = "1036998187880460289-K5YKVDBIlK5e1bsgsxzvmGk2z2BzTO"
        oauth_token_secret = "ogxeyO9mgweKrFEOZhn638EvTVvZjn06ynVwviShsVOzZ"
        return app_key, app_secret, oauth_token, oauth_token_secret
    elif key == 'v' or 'V':
        app_key = "d"
    else:
        print("Wrong identification key")

In [8]:
def cred_Processing(tweetsFile):
    idRegEx = re.compile(r".*ID=")
    endElRegEX = re.compile(r"'.*")
    output = open("outputTweets.json", "w")
    
    with open(tweetsFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            
            topicKey = topicData[0]
            topicTerms = topicData[1]
            topicTweetCount = topicData[2]
            tweetIdList = topicData[3]
            
            print(topicKey)
            
            realTweetsIds = []
            
            idElements = tweetIdList.split("),")
            for element in idElements:
                elArr = element.split(",")
                idEl = list(filter(lambda x: "ID" in x, elArr))[0]
                idEl = idRegEx.sub("", idEl)
                idEl = endElRegEX.sub("", idEl)
                
                realTweetsIds.append(int(idEl))
                
            realTweetsIds = list(set(realTweetsIds))
            
            topicMap = {
                "key" : topicKey,
                "terms" : topicTerms.split(","),
                "count" : topicTweetCount, 
                "tweets" : realTweetsIds
            }
            
            json.dump(topicMap, output, sort_keys=True)
            output.write("\n")
    output.close()

In [9]:
def rating_processing(ratingFile, tweetsFile):
    tweetsMap = {}
    
    with open(tweetsFile, "r") as f:
        for line in f:
            try:
                tweetData = json.loads(line)
                tweetsMap[tweetData["key"]] = tweetData
            except:
                print("Error")
            
    output = open(r"C:\Users\vdoan\Documents\College\SummerResearch\SmResearch\Data\CREDBANK\finalOutput.json", "w")
    
    with open(ratingFile, "r") as f:
        header = f.readline()
        
        for line in f:
            topicData = line.split("\t")
            topicKey = topicData[0]
            topicTerms = topicData[1]
            ratings = topicData[2]
            reasons = topicData[3]
            
            try:
                ratings = list(map(lambda x: int(x.strip().replace("'", "")), ratings.replace("[", "").replace("]", "").split(",")))
                ratings = np.array(ratings)
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Inserting ratings for {}".format(topicKey))
                tweetsMap[topicKey]["ratings"] = ratings.tolist()
                print("Success")
                print("Inserting mean for {}".format(topicKey))
                tweetsMap[topicKey]["mean"] = np.mean(ratings)
                print("Success")
                topicMap = tweetsMap[topicKey]
                print(topicMap["key"], topicMap["mean"])
                json.dump(topicMap, output, sort_keys=True)
                output.write("\n")
            except:
                print("{} is missing.".format(topicKey))
            
            
            
    output.close()

In [10]:
def binaryConversion(outputFile):
    labels = {}
    with open(outputFile, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            if(tweet['mean'] >= 1.9):
                labels[tweet['key']] = 1
            elif(tweet['mean'] <= 1.467):
                labels[tweet['key']] = 0
    return labels

In [11]:
#TODO: Retweet and date
def singleMessageFeatureExtraction(tweet_json):
    #print(tweet_json)
    retweets = 0
    try:
        retweets = tweet_json["tweet"]["retweet_status"]
        retweets = 1
    except:
        retweets = 0
    date = tweet_json["tweet"]["created_at"].split(" ")[0]
    dates = []
    if date == "Mon":
        dates = [1,0,0,0,0,0,0]
    elif date == "Tue":
        dates = [0,1,0,0,0,0,0]
    elif date == "Wed":
        dates = [0,0,1,0,0,0,0]
    elif date == "Thu":
        dates = [0,0,0,1,0,0,0]
    elif date == "Fri":
        dates = [0,0,0,0,1,0,0]
    elif date == "Sat":
        dates = [0,0,0,0,0,1,0]
    elif date == "Sun":
        dates = [0,0,0,0,0,0,1]
    return retweets, dates

In [12]:
def unshorten_url(url):
    return requests.head(url, allow_redirects=True).url

In [13]:
def structuralFeatureExtraction(topics, threshold):
    tweetsFeatures = None
    np.array(tweetsFeatures)
    count = 0
    for key in topics:
        print("Starting feature extraction for topic {} of {}".format(count, len(topics)))
        topicFeatures = []
        
        averageLength = 0
        frequencyHashtags = 0
        frequencyMedia = 0
        frequencyMentions = 0
        frequencyRetweets = 0
        frequencyWebLinks = 0
        
        for tweet in topics[key]:
            
            print("Extracting information from tweet " + str(tweet['id']))
            #Length of characters
            print("Adding Length of charactrs")
            averageLength = averageLength + len(tweet['tweet']['text'])
            
            #Frequency of Hashtags
            print("Adding frequency of Hashtags")
            frequencyHashtags = frequencyHashtags + len(tweet['tweet']['entities']['hashtags'])
                                
            #Frequency of Media
            print("Adding frequency of Media")
            #frequencyMedia = frequencyMedia + len(tweet['tweet']['entities']['media'])
            
            #Frequency of Mentions
            print("Adding Frequency of Mentions")
            frequencyMentions = frequencyMentions + len(tweet['tweet']['entities']['user_mentions'])
            
            #Frequency of Retweets
            print("Adding Frequency of Retweets")
            frequencyRetweets = frequencyRetweets
            
            #Frequency of Web Links
            print("Adding Frequency of Web Links")
            frequencyWebLinks = frequencyWebLinks + len(tweet['tweet']['entities']['urls'])
        
        #Appending features to list
        topicFeatures.append(len(topics[key]))
        topicFeatures.append(averageLength)
        topicFeatures.append(averageLength/len(topics[key]))
        topicFeatures.append(frequencyHashtags)
        topicFeatures.append(frequencyHashtags/len(topics[key]))
        topicFeatures.append(frequencyMedia)
        topicFeatures.append(frequencyMedia/len(topics[key]))
        topicFeatures.append(frequencyMentions)
        topicFeatures.append(frequencyMentions/len(topics[key]))
        topicFeatures.append(frequencyRetweets)
        topicFeatures.append(frequencyRetweets/len(topics[key]))
        topicFeatures.append(frequencyWebLinks)
        topicFeatures.append(frequencyWebLinks/len(topics[key]))
        
        #Appending feataures to features array
        topicFeatures = [topicFeatures]
        if(count == 0):
            tweetsFeatures = topicFeatures
            print("Length of array: {}".format(len(tweetsFeatures[count])))
        else:
            tweetsFeatures = np.append(tweetsFeatures, topicFeatures, axis = 0)
            print(tweetsFeatures.shape)
        
        print("Finished feature extraction for tweet {} of {}".format(count, len(tweets)))
        count += 1
        
    return tweetsFeatures

In [14]:
k  = np.array([1,2,3])

In [15]:
np.array([k])

array([[1, 2, 3]])

In [34]:
def contentFeatureExtraction(topics):
    
    tweetsFeatures = None
    np.array(tweetsFeatures)
    
    first = True
    first_pronouns = set(["i", "we"])
    second_pronouns = set(["you"])
    third_pronouns = set(["he", "him," "she", "her", "they", "them", "it"])
    
    count2 = 0
    tweetsFeatures = np.array([])
    #prop for proportion, fr for frequency 
    total_no_tweets = 0
        
    fr_quest_mark = 0
    fr_excl_mark = 0

    fr_emot_smile = 0 
        
    fr_tweets_hashtag = 0
        
    fr_pron_first = 0
    fr_pron_sec = 0
    fr_pron_third = 0
        
    #Polarity = Average sentiment score of tweets 
    avg_polarity = 0 
    avg_objectivity = 0
    disagreement = 0
    
 
    for topic in topics: 
        current_topic = topics[topic]
        total_no_tweets = len(current_topic)
        for tweet in current_topic: 
            split_tweet = tweet["tweet"]["text"].split()
            split_tweet_word = set(tweet["tweet"]["text"].split(" "))

            if '?' in split_tweet:
                fr_quest_mark += 1
            if '!' in split_tweet:
                fr_excl_mark += 1
            if '😊' in split_tweet:
                fr_emot_smile += 1
            if '#' in split_tweet:
                fr_tweets_hashtag += 1
            
            if (first_pronouns & split_tweet_word):
                fr_pron_first += 1
            if (second_pronouns & split_tweet_word):
                fr_pron_sec += 1
            if (third_pronouns & split_tweet_word):
                fr_pron_third += 1 
                
            
            tweetSentiment = TextBlob(tweet["tweet"]["text"])
            sentimentScore = tweetSentiment.sentiment[0]
            objectivityScore = tweetSentiment.sentiment[1]
            avg_polarity += sentimentScore
            avg_objectivity += objectivityScore
                
                
                
        prop_quest_mark = fr_quest_mark / total_no_tweets
        prop_excl_mark = fr_excl_mark / total_no_tweets
        prop_emot_smile = fr_emot_smile / total_no_tweets
        prop_tweets_hashtag = fr_tweets_hashtag / total_no_tweets

        
        prop_pron_first = fr_pron_first / total_no_tweets
        prop_pron_sec = fr_pron_sec / total_no_tweets
        prop_pron_third = fr_pron_third / total_no_tweets
        
        avg_polarity = avg_polarity / total_no_tweets
        avg_objectivity = avg_objectivity / total_no_tweets        
           
        disagreement = 0
        current_topic_feature = []
    
        current_topic_feature.append(avg_polarity)
        current_topic_feature.append(avg_objectivity)
        current_topic_feature.append(disagreement)
        current_topic_feature.append(fr_quest_mark)
        current_topic_feature.append(prop_quest_mark)
        current_topic_feature.append(fr_excl_mark)
        current_topic_feature.append(prop_excl_mark)
        current_topic_feature.append(fr_emot_smile)
        current_topic_feature.append(prop_emot_smile)
        current_topic_feature.append(fr_pron_first)
        current_topic_feature.append(prop_pron_first)
        current_topic_feature.append(fr_pron_sec)
        current_topic_feature.append(prop_pron_sec)
        current_topic_feature.append(fr_pron_third)
        current_topic_feature.append(prop_pron_third)
    
        topicFeatures = [current_topic_feature]
        if(count2 == 0):
            tweetsFeatures = topicFeatures
            print("Length of array: {}".format(len(tweetsFeatures[count2])))
        else:
            tweetsFeatures = np.append(tweetsFeatures, topicFeatures, axis = 0)
            print(tweetsFeatures.shape)
        count2 = count2 + 1
    
    return tweetsFeatures

In [71]:
def userFeatureExtraction(topics):
    tweetsFeatures = None
    np.array(tweetsFeatures)
    
    first = True
    userFeatures = None
    count2 = 0
    tweetsFeatures = np.array([])
    #Avg date create
    avg_date_user_create = 0
        
    #Average follower count
    avg_follower_cnt = 0
    
    #Average friend count
    avg_friend_cnt = 0 
    
    #Average authored status count 
    avg_aut_stat_cnt = 0
    
    #Frequency of verified author 
    fr_veri_aut = 0 
    
    #Author of the First Tweet is Verified
    fr_first_veri = 0
    
        
    avg_date_tweet_create = 0

    for topic in topics:
        no_tweet = len(topic)
        for tweet in topics[topic]:
            
            #Average account age format : "created_at": "Fri Jun 26 07:47:06 +0000 2009"
            user_created_string = tweet['tweet']['user']['created_at']
            tweet_created_string = tweet['tweet']['created_at']
            user_created_string_list = user_created_string.split(" ")
            tweet_created_string_list = tweet_created_string.split(" ")
            day_user = user_created_string_list[2]
            month_user = 0
            year_user = user_created_string_list[5]
            
            day_tweet = tweet_created_string_list[2]
            month_tweet = 0
            year_tweet = tweet_created_string_list[5]

            if (user_created_string_list[1] == 'Jan'):
                month_user = 1
            elif (user_created_string_list[1] == 'Feb'):
                month_user = 2
            elif (user_created_string_list[1] == 'Mar'):
                month_user = 3
            elif (user_created_string_list[1] == 'Apr'):
                month_user = 4
            elif (user_created_string_list[1] == 'May'):
                month_user = 5
            elif (user_created_string_list[1] == 'Jun'):
                month_user = 6
            elif (user_created_string_list[1] == 'Jul'):
                month_user = 7
            elif (user_created_string_list[1] == 'Aug'):
                month_user = 8
            elif (user_created_string_list[1] == 'Sep'):
                month_user = 9
            elif (user_created_string_list[1] == 'Oct'):
                month_user = 10
            elif (user_created_string_list[1] == 'Nov'):
                month_user = 11
            elif (user_created_string_list[1] == 'Dec'):
                month_user = 12
                
                
            if (tweet_created_string_list[1] == 'Jan'):
                month_tweet = 1
            elif (tweet_created_string_list[1] == 'Feb'):
                month_tweet = 2
            elif (tweet_created_string_list[1] == 'Mar'):
                month_tweet = 3
            elif (tweet_created_string_list[1] == 'Apr'):
                month_tweet = 4
            elif (tweet_created_string_list[1] == 'May'):
                month_tweet = 5
            elif (tweet_created_string_list[1] == 'Jun'):
                month_tweet = 6
            elif (tweet_created_string_list[1] == 'Jul'):
                month_tweet = 7
            elif (tweet_created_string_list[1] == 'Aug'):
                month_tweet = 8
            elif (tweet_created_string_list[1] == 'Sep'):
                month_tweet = 9
            elif (tweet_created_string_list[1] == 'Oct'):
                month_tweet = 10
            elif (tweet_created_string_list[1] == 'Nov'):
                month_tweet = 11
            elif (tweet_created_string_list[1] == 'Dec'):
                month_tweet = 12
            
            user_created_date = date(int(year_user),int(month_user),int(day_user))
            tweet_created_date = date(int(year_tweet),int(month_tweet),int(day_tweet))
            current_date = date.today()
            user_delta = current_date - user_created_date
            tweet_delta = current_date - tweet_created_date
            avg_date_user_create += user_delta.days
            avg_date_tweet_create += tweet_delta.days
            
            
            
            follower_count = tweet['tweet']['user']['followers_count']
        
            friend_count = tweet['tweet']['user']['friends_count']
        
            aut_stat_count = tweet['tweet']['user']['statuses_count']
        
            #0 for false, 1 for true
            is_verified = 0
        
            if (tweet['tweet']['user']['verified'] == True):
                is_verified = 1
            else:
                is_verified = 0
            
            if (tweet['tweet']['in_reply_to_status_id'] != None) and (tweet['tweet']['user']['verified'] == True):
                fr_first_veri += 1
            
                                                                      
            avg_follower_cnt += follower_count
            avg_friend_cnt += friend_count
            avg_aut_stat_cnt += aut_stat_count
            fr_veri_aut += is_verified
                                     
        G = nx.Graph()
        current_users = []
                                                                    
        for tweet in topics[topic]:
            if tweet['tweet']['user']['id'] not in current_users:
                current_users.append(tweet['tweet']['user']['id'])
                G.add_node(tweet['tweet']['user']['id'])                  
                                                                    
        for tweet in topics[topic]:
            for i in tweet['tweet']['entities']['user_mentions']:
                mention_id = i['id']
                G.add_edge(tweet['tweet']['user']['id'], mention_id)
    
            if 'retweeted_status' in i:
                retweet_id = i['tweet']['retweeted_status']['id']
                G.add_edge(tweet['tweet']['user']['id'], retweet_id)                                        
                                                                    
                                                                      
        avg_date_user_create = avg_date_user_create / no_tweet                                                              
        avg_follower_cnt = avg_follower_cnt / no_tweet
        avg_friend_cnt = avg_friend_cnt / no_tweet
        avg_aut_stat_cnt = avg_aut_stat_cnt / no_tweet
        avg_date_tweet_create = avg_date_tweet_create / no_tweet                                                    
        
        current_topic_feature = []
        
        current_topic_feature.append(avg_date_user_create)                                                              
        current_topic_feature.append(avg_follower_cnt)
        current_topic_feature.append(avg_friend_cnt)
        current_topic_feature.append(avg_aut_stat_cnt)
        current_topic_feature.append(fr_veri_aut)
        current_topic_feature.append(fr_first_veri)
        current_topic_feature.append(density(G)) 

        print("Current topic extracted features: ")
        print(avg_date_user_create)
        print(avg_follower_cnt)
        print(avg_friend_cnt)
        print(avg_aut_stat_cnt)
        print(fr_veri_aut)
        print(fr_first_veri)
        print(density(G))
        print()
        
        topicFeatures = [current_topic_feature]
        if(count2 == 0):
            tweetsFeatures = topicFeatures
            print("Length of array: {}".format(len(tweetsFeatures[count2])))
        else:
            tweetsFeatures = np.append(tweetsFeatures, topicFeatures, axis = 0)
            print(tweetsFeatures.shape)
        count2 = count2 + 1
    
    return tweetsFeatures

In [18]:
def temporalFeatureExtraction(topics):
    
    counts = 1
    count = 1
    first = True
    firstArray = True
    temporalFeatures = []
    tweetsFeatures = np.array([])
    count2 = 0
    #first need to sort the tweets
    for k in topics:
        print("Starting feature extraction for topic {} of {}".format(counts, len(topics)))
    
        #Sort tweets
        sortedArray = sorted(topics[k], key=lambda x: datetime.strptime(x['tweet']['created_at'],"%a %b %d %H:%M:%S +0000 %Y"))
        time = 0
        
        followers = 0
        friends = 0
        status = 0
        perMin = 0
        
        for tweet in sortedArray:
            if(first == True):
                time = timeConvert(tweet['tweet']['created_at'])
#             print("Beginning Time: " + str(time))
#             print("Current Time: " + str(timeConvert(tweet['tweet']['created_at'])) + ", In seconds: " + str(timeConvert(tweet['tweet']['created_at']) - time))
#             print("Time difference: " + str(timeConvert(tweet['tweet']['created_at']) - time))
            if((timeConvert(tweet['tweet']['created_at']) - time) < 60 and (timeConvert(tweet['tweet']['created_at']) >= time)):
                print("In section: " + str(count))
                print("Time difference: " + str(timeConvert(tweet['tweet']['created_at']) - time))
                print(tweet['tweet']['created_at'])
                first = False
            elif(timeConvert(tweet['tweet']['created_at']) > time):
                if(((time + timeConvert(tweet['tweet']['created_at'])) - time) < 60) :
                    print("In section: " + str(count))
                    print("Time difference: " + str(timeConvert(tweet['tweet']['created_at']) - time))
                    print(tweet['tweet']['created_at'])
                    first = False
                else:
                    count = count + 1
                    print("Start of Section: " + str(count))
                    print()
                    print("Time difference for change is due to: " + str(timeConvert(tweet['tweet']['created_at']) - time))
                    time = timeConvert(tweet['tweet']['created_at'])

                    print(tweet['tweet']['created_at'])
                    print("Time difference: " + str(timeConvert(tweet['tweet']['created_at']) - time))
            else:
                count = count + 1
                print("Start of Section: " + str(count))
                print()
                print("Time difference for change is due to: " + str(timeConvert(tweet['tweet']['created_at']) - time))
                time = timeConvert(tweet['tweet']['created_at'])

                print(tweet['tweet']['created_at'])
                print("Time difference: " + str(timeConvert(tweet['tweet']['created_at']) - time))
            
            #Account Age
            
            #Difference Between account age and tweet publication time
            
            #Author's Follower
            followers = followers + tweet['tweet']['user']["followers_count"]
            print("Current Followers: " + str(followers))
            #Author's Friends
            friends = friends + tweet['tweet']['user']["friends_count"]
            print("Current Friends: " + str(friends))
            #Author's Statuses
            if(tweet['tweet']['user']["verified"] == "True"):
                status = status + 1
            print("Current Amount of Statusese: " + str(status))
            #Number of tweets per minute
            perMin = perMin + 1
            print("Current Amount of Tweets: " + str(perMin))
        
        counts = counts + 1
        current_topic_features = []
        current_topic_features.append(followers)
        current_topic_features.append(friends)
        current_topic_features.append(status)
        current_topic_features.append(perMin)
        print(current_topic_features)
        
        topicFeatures = [current_topic_features]
        if(count2 == 0):
            tweetsFeatures = topicFeatures
            print("Length of array: {}".format(len(tweetsFeatures[count2])))
        else:
            tweetsFeatures = np.append(tweetsFeatures, topicFeatures, axis = 0)
            print(tweetsFeatures.shape)
        count2 = count2 + 1
#         if(firstArray == True):
#             temporalFeatures = np.array([current_topic_features])
#             firstArray = False
#             print("First time")
#         else:
#             print("Other times")
#             features = np.array([current_topic_features])
#             temporalFeatures = np.concatenate((temporalFeatures, features), axis=1)
        
        #print("Shape of features: " + str(temporalFeatures.size))
            
    
    return tweetsFeatures

In [73]:
def retrieveFeatures(topics, threshold):
    structuralArray = structuralFeatureExtraction(topics, threshold)
    contentArray = contentFeatureExtraction(topics)
    userArray = userFeatureExtraction(topics)
    temporalArray = temporalFeatureExtraction(topics)
    
    print("contentArray: ", len(contentArray[0]))
    print("structuralArray: ", len(structuralArray[0]))
    print("userArray: ", len(userArray[0]))
    print("temporalArray: ", len(temporalArray[0]))
    featureArray = np.hstack((contentArray, structuralArray))
    print("featureArray: ", featureArray.shape)
    featureArray_2 = np.hstack((featureArray, userArray))
    print("featureArray: ", featureArray_2.shape)
    featureArray_3 = np.hstack((featureArray_2, temporalArray))
    print("featureArray: ", featureArray_3.shape)
    print("Feature Created.")
    return featureArray_3
    

In [20]:
def timeConvert(time):
    time = datetime.strptime(time, "%a %b %d %H:%M:%S +0000 %Y")
    return timedelta(minutes = time.minute).seconds + time.second

In [21]:
#             print("Adding words")
#             #Length of words 2
#             indiTweetFeature.append(len(tweet.split(" ")))
        
#             print("Adding question mark")
#             #Contains question mark 3
#             results = collections.Counter(tweet)
#             if(results['?'] > 1):
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)

#             print('Adding exclamation mark')
#             #Contains exclamation 4
#             if(results['!'] > 1):
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)

#             print("Adding smiles")
#             #Contains emoticon smile 5
#             emojis = "😊"
#             indiTweetFeature.append(tweet.count(emojis))

#             print("!?")
#             #Contains multiple question or exclamation marks 6
#             if results['?'] and results['!'] > 1:
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)

#             print("Adding first person pronouns")
#             #Contains pronoun first, sceond, third 7
#             first_pronouns = ["i", "we"]
#             tweetSplit = tweet.split(" ")
#             for x in range(len(tweetSplit)):
#                 if(tweetSplit[x].lower() in first_pronouns):
#                     indiTweetFeature.append(1)
#                     break;
#                 if(x == len(tweetSplit)-1):
#                     indiTweetFeature.append(0)

#             print("Adding second person pronouns")
#             second_pronouns = ["you"]
#             tweetSplit = tweet.split(" ")
#             for x in range(len(tweetSplit)):
#                 if(tweetSplit[x].lower() in second_pronouns):
#                     indiTweetFeature.append(1)
#                     break;
#                 if(x == len(tweetSplit)-1):
#                     indiTweetFeature.append(0)

#             print("Adding third person pronouns")
#             third_pronouns = ["he", "him," "she", "her", "they", "them", "it"]

#             tweetSplit = tweet.split(" ")
#             for x in range(len(tweetSplit)):
#                 if(tweetSplit[x].lower() in third_pronouns):
#                     indiTweetFeature.append(1)
#                     break;
#                 if(x == len(tweetSplit)-1):
#                     indiTweetFeature.append(0)

#             print("Adding uppercase")
#             #Count uppercase letters 8
#             upperLetterCount = 0
#             for letter in tweet:
#                 if(letter.isupper()):
#                     upperLetterCount += 1
#             indiTweetFeature.append(upperLetterCount)

#             print("Adding url")
#             #Number of URLs 9
#             url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet) 
#             indiTweetFeature.append(len(url))
        
#             #Contains popular domain top 100,1000,10000
#             print("Adding popular domain")
#             file = pd.read_csv('/media/eric/TOSHIBA EXT/University/Research/Data/top-1m.csv')
#             try:
#                 urls = tweet_json[count]['tweet']['entities']['urls']
#             except:
#                 print(tweet_json[count]['tweet'])
#             top100 = 0
#             top1000 = 0
#             top10000 = 0

#             for url in urls:
#                 urlParsed = None
#                 rank = None
#                 try:
#                     split_url = urlsplit(url['expanded_url'])   
#                     shortUrl = ''
#                     if(split_url.netloc[:4] == 'www.'):
#                         shortUrl = split_url.netloc[4:]
#                     else:
#                         shortUrl = split_url.netloc
#                     print(shortUrl)
#                     print(file.loc[file['site'] == shortUrl]['rank'])
#                     rank = int(file.loc[file['site'] == split_url.netloc]['rank'])
#                 except Exception as e:
#                     rank = 10001
#                 if rank <= 100:
#                     top100 = 1
#                 if rank <= 1000:
#                     top1000 = 1
#                 if rank <= 10000:
#                     top10000 = 1
        
#             indiTweetFeature.append(top100)
#             indiTweetFeature.append(top1000)
#             indiTweetFeature.append(top10000)
                
#             print("Adding mention")
#             #Contains user mention 10
#             result = re.findall("(^|[^@\w])@(\w{1,15})", tweet)
#             if(len(result) >= threshold):
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)
    
#             # Length of hashtags 11
#             print("Adding hashtags")
#             hashtags = [i  for i in tweet.split() if i.startswith("#") ]
#             indiTweetFeature.append(len(hashtags))
     
#             #Contains stock symbol 12
#             print("Addings stocks")
#             file = open('/media/eric/TOSHIBA EXT/University/Research/Data/nasdaqlisted.txt', 'r')
#             filecontents = file.readlines()
#             stock_symbols = []
#             for line in filecontents:
#                 splitline = line.split("|")
#                 stock_symbols.append(splitline[0])
            
#             del stock_symbols[0]
        
#             stockCount = 0
#             for word in tweetSplit:
#                 if(word in stock_symbols):
#                     stockCount += 1
        
#             if(stockCount > 0):
#                 indiTweetFeature.append(1)
#             else:
#                 indiTweetFeature.append(0)
                
#             retweets, date = singleMessageFeatureExtraction(tweet_json[count])
#             print("Adding retweets")
#             indiTweetFeature.append(retweets)
            
#             print("Adding date")
#             indiTweetFeature = indiTweetFeature + date
        
#             print("Adding sentiment")
#             #Sentiment 13, 14, 15
#             positiveWords = 0
#             negativeWords= 0
    
    
#             tweetSentiment = TextBlob(tweet)
#             sentimentScore = tweetSentiment.sentiment[0]
    
#             for word in tweetSentiment:
#                 word = TextBlob(word)
#                 if(word.sentiment.polarity) > 0:
#                     positiveWords = positiveWords + 1
#                 elif(word.sentiment.polarity) < 0:
#                     negativeWords = negativeWords + 1
#             indiTweetFeature.append(positiveWords)
#             indiTweetFeature.append(negativeWords)
#             indiTweetFeature.append(sentimentScore)